In [1]:
import sys
import warnings
sys.path.append("../Pythonscripts")
sys.path.append(r"C:\Users\atedi\OneDrive\Documents\SageBionetworks\synapse\Scripts")
import pandas as pd
import numpy as np
import synapseclient as sc
from pdkit_features import pdkit_pipeline
from myutils import get_synapse_table, get_acceleration_ts, open_filepath
from spectral_flatness import get_spectrum
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pdkit
%matplotlib inline
warnings.simplefilter("ignore")

In [4]:
syn = sc.login()

Welcome, aryton tediarjo!



INFO:synapseclient_default:Welcome, aryton tediarjo!



In [52]:

def clean_accelerometer_data(data):
    data = data.dropna(subset = ["x", "y", "z"])
    date_series = pd.to_datetime(data["timestamp"], unit = "s")
    data["td"] = date_series - date_series.iloc[0]
    data["td"] = data["td"].apply(lambda x: x.total_seconds())
    data["time"] = data["td"]
    data = data.set_index("time")
    data.index = pd.to_datetime(data.index, unit = "s")
    data["AA"] = np.sqrt(data["x"]**2 + data["y"]**2 + data["z"]**2)
    return data

## function to retrieve sensors ## 
def get_sensor_types(filepath):
    data = open_filepath(filepath)
    if "sensorType" in data.columns:
        return data["sensorType"].dropna().unique()
    else:
        return np.NaN

## function to retrieve unit of measurements ##
def get_units(filepath):
    data = open_filepath(filepath)
    if "sensorType" in data.columns:
        return data["unit"].dropna().unique()
    else:
        return np.NaN

In [53]:
data = get_synapse_table(syn, ["ae8fe177-8e0f-4cd7-a9eb-bd5a82ac02d3", 
                               "4b1faed8-9f46-457d-b3cc-e158c2af4f65",
                              "86aaa92b-e31e-44ac-b1bc-3c68bd118652"], "syn12514611")

INFO:synapseclient_default:Downloading 0 files, 28 cached locally


In [54]:
data["sensor_walking"] = data["walk_motion.json_pathfile"].apply(get_sensor_types)
data["units"] = data["walk_motion.json_pathfile"].apply(get_units)
# data["sensorsUsed_balanced"] = data["balance_motion.json_pathfile"].apply(get_sensors)

Index(['eventAccuracy', 'sensor', 'sensorType', 'timestamp', 'timestampDate',
       'unit', 'uptime', 'x', 'y', 'z'],
      dtype='object')
Index(['eventAccuracy', 'sensor', 'sensorType', 'timestamp', 'timestampDate',
       'unit', 'uptime', 'x', 'y', 'z'],
      dtype='object')
Index(['eventAccuracy', 'sensor', 'sensorType', 'timestamp', 'timestampDate',
       'unit', 'uptime', 'x', 'y', 'z'],
      dtype='object')
Index(['eventAccuracy', 'sensor', 'sensorType', 'timestamp', 'timestampDate',
       'unit', 'uptime', 'x', 'y', 'z'],
      dtype='object')
Index(['eventAccuracy', 'sensor', 'sensorType', 'timestamp', 'timestampDate',
       'unit', 'uptime', 'x', 'y', 'z'],
      dtype='object')
Index(['stepPath', 'timestamp', 'timestampDate', 'uptime'], dtype='object')
Index(['eventAccuracy', 'sensor', 'sensorType', 'timestamp', 'timestampDate',
       'unit', 'uptime', 'x', 'y', 'z'],
      dtype='object')
Index(['eventAccuracy', 'sensor', 'sensorType', 'timestamp', 'timestampDate',


In [55]:
data

,recordId,phoneInfo,createdOn,healthCode,walk_motion.json_path_id,balance_motion.json_path_id,walk_motion.json_pathfile,balance_motion.json_pathfile,sensor_walking,units
0,c334101d-a888-414d-8df2-5f97b46c53ca,Google Android SDK built for x86,2018-11-18 23:31:21.661,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34108788.0,34108790.0,c:/users/atedi/.synapsecache/788/34108788/c334...,c:/users/atedi/.synapsecache/790/34108790/c334...,"[rotationRate, acceleration]","[rad/s, g]"
1,6f378cf7-4005-4728-a089-a70d2fd99b2b,Google Android SDK built for x86,2018-11-19 02:14:21.893,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34108794.0,34108795.0,c:/users/atedi/.synapsecache/794/34108794/6f37...,c:/users/atedi/.synapsecache/795/34108795/6f37...,"[acceleration, rotationRate]","[g, rad/s]"
2,efe03b98-cae0-4adc-9242-447b00d68e57,Google Android SDK built for x86,2018-11-20 19:34:06.890,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34129488.0,34129490.0,c:/users/atedi/.synapsecache/488/34129488/efe0...,c:/users/atedi/.synapsecache/490/34129490/efe0...,"[acceleration, rotationRate]","[g, rad/s]"
3,d39344fa-cb4b-4a7b-ab38-ce36f5238c27,Google Android SDK built for x86,2018-11-20 19:55:10.626,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34129497.0,34129498.0,c:/users/atedi/.synapsecache/497/34129497/d393...,c:/users/atedi/.synapsecache/498/34129498/d393...,"[acceleration, rotationRate]","[g, rad/s]"
4,adb9029d-ae0b-4f39-91eb-6d9166241e98,Google Android SDK built for x86,2018-11-21 22:13:15.571,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34164035.0,34164039.0,c:/users/atedi/.synapsecache/35/34164035/adb90...,c:/users/atedi/.synapsecache/39/34164039/adb90...,"[acceleration, rotationRate]","[g, rad/s]"
5,526e2401-1ba8-45d9-82b7-a2f9ac0b9fa6,iPhone Simulator; iOS/12.1,2018-11-23 22:21:53.998,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34191272.0,34191273.0,c:/users/atedi/.synapsecache/272/34191272/526e...,c:/users/atedi/.synapsecache/273/34191273/526e...,NaN,NaN
6,04d89185-6495-46c0-bf8a-c759308642c7,Google Android SDK built for x86,2018-11-23 22:22:21.608,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34191274.0,34191278.0,c:/users/atedi/.synapsecache/274/34191274/04d8...,c:/users/atedi/.synapsecache/278/34191278/04d8...,"[rotationRate, acceleration]","[rad/s, g]"
7,a6d01b46-23f7-4e91-bb0c-7fef5ee95642,Google Android SDK built for x86,2018-11-23 22:41:11.072,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34191275.0,34191279.0,c:/users/atedi/.synapsecache/275/34191275/a6d0...,c:/users/atedi/.synapsecache/279/34191279/a6d0...,"[rotationRate, acceleration]","[rad/s, g]"
8,eee210a4-2454-48c4-8be1-01dd82cc6512,Motorola moto x4,2018-11-24 16:42:59.614,86aaa92b-e31e-44ac-b1bc-3c68bd118652,34193326.0,34193330.0,c:/users/atedi/.synapsecache/326/34193326/eee2...,c:/users/atedi/.synapsecache/330/34193330/eee2...,"[acceleration, rotationRate]","[g, rad/s]"
9,35dc4c0b-6ef6-4b1e-826f-6d84742032f4,Motorola moto x4,2018-11-27 18:06:48.887,86aaa92b-e31e-44ac-b1bc-3c68bd118652,34283598.0,34283601.0,c:/users/atedi/.synapsecache/598/34283598/35dc...,c:/users/atedi/.synapsecache/601/34283601/35dc...,"[rotationRate, acceleration]","[rad/s, g]"


In [58]:
var = "AA"
gp = pdkit.GaitProcessor(duration=data.td[-1])
data = gp.resample_signal(data)
    ### instantiate empty dictionary ###
feature_dict = {}
try:  
    no_of_steps = gp.gait(data[var])[0]
except:
    no_of_steps = 0
try:
    gait_step_regularity, gait_stride_regularity, gait_symmetry = gp.gait_regularity_symmetry(data[var])
except:
    gait_step_regularity = 0
    gait_stride_regularity = 0
    gait_symmetry = 0
try:
    frequency_of_peaks = gp.frequency_of_peaks(data[var])
except:
    frequency_of_peaks = 0
try:
    freeze_index = gp.freeze_of_gait(data[var])[1]
except:
    freeze_index = 0
try:
    freeze_count = sum(i > 2.0 for i in freeze_index)
except:
    freeze_count = 0
try:
    speed_of_gait = gp.speed_of_gait(data[var], wavelet_level = 6)
except:
    speed_of_gait = 0
        
### fill in values to each keys
feature_dict["no_of_steps_".format(var)] = no_of_steps
feature_dict["mean_freeze_index_".format(var)] = np.mean(freeze_index)
feature_dict["median_freeze_index_".format(var)] = np.median(freeze_index)
feature_dict["max_freeze_index_".format(var)] = np.max(freeze_index)
feature_dict["count_freeze_index_".format(var)] = freeze_count
feature_dict["speed_of_gait_".format(var)] = speed_of_gait
feature_dict["gait_step_regularity_".format(var)] = gait_step_regularity
feature_dict["gait_stride_regularity_".format(var)] = gait_stride_regularity
feature_dict["gait_symmetry_".format(var)] = gait_symmetry
feature_dict["frequency_of_peaks_".format(var)] = frequency_of_peaks
        
    ## Final Check to clear nan values to zero ##
for k, v in feature_dict.items():
    if np.isnan(v):
        feature_dict[k] = 0

Sampling rate is 99.33333333333333 Hz
Sampling rate is 99.33333333333333 Hz


In [59]:
feature_dict

{'no_of_steps_': 0,
 'mean_freeze_index_': 8.2914,
 'median_freeze_index_': 3.0395563,
 'max_freeze_index_': 117.14901,
 'count_freeze_index_': 36,
 'speed_of_gait_': 0.031323065007508595,
 'gait_step_regularity_': 0,
 'gait_stride_regularity_': 0,
 'gait_symmetry_': 0,
 'frequency_of_peaks_': 0.8908512228521506}

In [56]:
data

,timestamp,uptime,x,y,z,td,AA
time,,,,,,,
1970-01-01 00:00:00.000,0.086895,7177.766841,-0.065855,-0.918915,-0.401382,0.000000,1.003714
1970-01-01 00:00:00.010,0.096919,7177.776865,-0.044920,-0.910599,-0.397156,0.010024,0.993706
1970-01-01 00:00:00.020,0.106960,7177.786906,-0.034681,-0.906677,-0.394943,0.020065,0.989043
1970-01-01 00:00:00.030,0.117011,7177.796957,-0.030088,-0.906342,-0.393860,0.030116,0.988255
1970-01-01 00:00:00.040,0.127042,7177.806988,-0.053099,-0.917908,-0.397034,0.040147,1.000581
1970-01-01 00:00:00.050,0.137073,7177.817019,-0.066343,-0.921478,-0.402405,0.050178,1.006491
1970-01-01 00:00:00.060,0.147106,7177.827052,-0.064909,-0.918045,-0.398666,0.060211,1.001793
1970-01-01 00:00:00.070,0.157138,7177.837084,-0.040281,-0.910553,-0.395752,0.070243,0.993007
1970-01-01 00:00:00.080,0.167169,7177.847115,-0.036405,-0.907639,-0.392914,0.080274,0.989140
